In [1]:
!pip install -qq -e git+http://github.com/tensorflow/cleverhans.git#egg=cleverhans
import sys
sys.path.append('/content/src/cleverhans')
!pip install keras

Using TensorFlow backend.


In [0]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.loss import LossCrossEntropy
from cleverhans.utils import AccuracyReport
from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import train, model_eval
import keras
from keras import backend
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import flags
import os
import matplotlib.pyplot as plt

In [0]:
sess = tf.Session()
keras.backend.set_session(sess)

x_train, y_train, x_test, y_test = data_mnist()
report = AccuracyReport()

In [0]:
 img_rows, img_cols, nchannels = x_train.shape[1:4]
 nb_classes = y_train.shape[1]

In [0]:
    # Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                          nchannels))
y = tf.placeholder(tf.float32, shape=(None, nb_classes))


In [24]:
model = cnn_model(img_rows=img_rows, img_cols=img_cols,
                      channels=nchannels, nb_filters=64,
                      nb_classes=nb_classes)
preds = model(x)
print("Defined TensorFlow model graph.")

Defined TensorFlow model graph.


In [0]:
wrap = KerasModelWrapper(model)

In [0]:
def evaluate():
        # Evaluate the accuracy of the MNIST model on legitimate test examples
  eval_params = {'batch_size': 128}
  acc = model_eval(sess, x, y, preds, x_test, y_test, args=eval_params)
  report.clean_train_clean_eval = acc
#        assert X_test.shape[0] == test_end - test_start, X_test.shape
  print('Test accuracy on legitimate examples: %0.4f' % acc)

In [0]:
train_params = {
        'nb_epochs': 6,
        'batch_size': 128,
        'learning_rate': 0.001
    }


In [28]:
loss = LossCrossEntropy(wrap, smoothing=0.1)
train(sess, loss, x, y, x_train, y_train, evaluate=evaluate,
              args=train_params)

Test accuracy on legitimate examples: 0.9866
Test accuracy on legitimate examples: 0.9914
Test accuracy on legitimate examples: 0.9927
Test accuracy on legitimate examples: 0.9924
Test accuracy on legitimate examples: 0.9923
Test accuracy on legitimate examples: 0.9939


True

In [0]:
eval_params = {'batch_size': 128}
acc = model_eval(sess, x, y, preds, x_train, y_train, args=eval_params)
report.train_clean_train_clean_eval = acc

In [0]:
fgsm = FastGradientMethod(wrap, sess=sess)
fgsm_params = {'eps': 0.3,
                'clip_min': 0.,
                'clip_max': 1.}

In [0]:
adv_x = fgsm.generate(x, **fgsm_params)
    # Consider the attack to be constant
adv_x = tf.stop_gradient(adv_x)
preds_adv = model(adv_x)

In [35]:
    # Evaluate the accuracy of the MNIST model on adversarial examples
eval_par = {'batch_size': 128}
acc = model_eval(sess, x, y, preds_adv, x_test, y_test, args=eval_par)
print('Test accuracy on adversarial examples: %0.4f\n' % acc)
report.clean_train_adv_eval = acc

Test accuracy on adversarial examples: 0.1126

